## 네이버 영화 리뷰
### 데이터 불러오기

참고 :  https://wikidocs.net/44249

In [1]:
import pandas as pd

train_data= pd.DataFrame(pd.read_csv('ratings_train.txt', sep='\t', quoting=3))
test_data = pd.DataFrame(pd.read_csv('ratings_test.txt', sep='\t', quoting=3))

print(train_data.shape)
print(test_data.shape)

(150000, 3)
(50000, 3)


### 데이터 정제하기

In [2]:
train_data['document'].nunique(), train_data['label'].nunique() #중복 개수 약 4000개

(146182, 2)

In [3]:
train_data.drop_duplicates(subset=['document'], inplace=True) 
# document 열에서 중복인 내용이 있다면 중복 제거

In [4]:
print('총 샘플의 수 :',len(train_data))

총 샘플의 수 : 146183


In [5]:
print(train_data.isnull().values.any()) # null 값 확인

True


In [6]:
print(train_data.isnull().sum())

id          0
document    1
label       0
dtype: int64


In [7]:
# train_data.loc[train_data.document.isnull()]

In [8]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [9]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
train_data[:5]

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [10]:
import numpy as np
# 한글이 없는 리뷰였다면 이제 더 이상 아무런 값도 없는 빈(empty) 값이 되었을 것입니다.
# train_data에 빈 값을 가진 행이 있다면 Null 값으로 변경하도록 하고,
# 다시 한 번 Null 값이 존재하는지 확인해보겠습니다.
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

id            0
document    391
label         0
dtype: int64


In [11]:
#Null 제거
train_data = train_data.dropna(how = 'any')
print(len(train_data))

145791


In [12]:
# 테스트 데이터에도 전처리 하기

test_data.drop_duplicates(subset = ['document'], inplace=True) 
# document 열에서 중복인 내용이 있다면 중복 제거

In [13]:
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") 
# 정규 표현식 수행

In [14]:
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
test_data

,id,document,label
0,6270596,굳 ㅋ,1
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
5,7898805,음악이 주가 된 최고의 음악영화,1
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
49997,9072549,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


In [15]:
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

전처리 후 테스트용 샘플의 개수 : 48995


### 토큰화

In [16]:
# Mecab
from konlpy.tag import Mecab
# 선언
mecab= Mecab()

In [17]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과',
             '도','를','으로','자','에','와','한','하다']

In [18]:
X_train = []
for sentence in train_data['document']:
    temp_x = []
    temp_x = mecab.morphs(sentence) # 토큰화
    temp_x = [word for word in temp_x if not word in stopwords] # 불용어제거
    X_train.append(temp_x)

In [19]:
print(X_train[:3])

[['아', '더', '빙', '진짜', '짜증', '나', '네요', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍', '지', '않', '구나'], ['너무', '재', '밓었다그래서보는것을추천한다']]


In [20]:
train_data['tokens'] = X_train
train_data.head()

,id,document,label,tokens
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,"[아, 더, 빙, 진짜, 짜증, 나, 네요, 목소리]"
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1,"[흠, 포스터, 보고, 초딩, 영화, 줄, 오버, 연기, 조차, 가볍, 지, 않, 구나]"
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[너무, 재, 밓었다그래서보는것을추천한다]"
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0,"[교도소, 이야기, 구먼, 솔직히, 재미, 없, 다, 평점, 조정]"
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1,"[사이몬페그, 익살, 스런, 연기, 돋보였, 던, 영화, 스파이더맨, 에서, 늙, ..."


In [21]:
train_data = train_data.reset_index(drop = True)

In [22]:
token =[]
for i in range(len(train_data)):
    tokens= str(train_data['tokens'][i]).replace('[','').replace(']','').replace("'","").replace(',',' ')
    token.append(tokens)
train_data['tokens'] = token
train_data

,id,document,label,tokens
0,9976970,아 더빙 진짜 짜증나네요 목소리,0,아 더 빙 진짜 짜증 나 네요 목소리
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍 지 않 구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무 재 밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0,교도소 이야기 구먼 솔직히 재미 없 다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1,사이몬페그 익살 스런 연기 돋보였 던 영화 스파이더맨 에서 늙 어...
...,...,...,...,...
145786,6222902,인간이 문제지 소는 뭔죄인가,0,인간 문제지 소 뭔 죄 인가
145787,8549745,평점이 너무 낮아서,1,평점 너무 낮 아서
145788,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0,이게 뭐 요 한국인 거들먹거리 고 필리핀 혼혈 착하 다
145789,2376369,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1,청춘 영화 최고봉 방황 우울 했 던 날 자화상


In [23]:
X_test = []
for sentence in test_data['document']:
    temp_x = []
    temp_x = mecab.morphs(sentence) # 토큰화
    temp_x = [word for word in temp_x if not word in stopwords] # 불용어제거
    X_test.append(temp_x)

In [24]:
print(X_test[:3])

[['굳', 'ㅋ'], ['뭐', '야', '평점', '나쁘', '진', '않', '지만', '점', '짜리', '더더욱', '아니', '잖아'], ['지루', '하', '지', '않', '은데', '완전', '막장', '임', '돈', '주', '고', '보', '기']]


In [25]:
len(test_data),len(X_test)

(48995, 48995)

In [26]:
test_data['tokens'] = X_test
test_data.head()

,id,document,label,tokens
0,6270596,굳 ㅋ,1,"[굳, ㅋ]"
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,"[뭐, 야, 평점, 나쁘, 진, 않, 지만, 점, 짜리, 더더욱, 아니, 잖아]"
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,"[지루, 하, 지, 않, 은데, 완전, 막장, 임, 돈, 주, 고, 보, 기]"
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,"[만, 아니, 었, 어도, 별, 다섯, 개, 줬, 을, 텐데, 왜, 로, 나와서, ..."
5,7898805,음악이 주가 된 최고의 음악영화,1,"[음악, 주, 된, 최고, 음악, 영화]"


In [27]:
test_data = test_data.reset_index(drop = True)
test_data['tokens'][0]
str(test_data['tokens'][0]).replace('[','').replace(']','').replace("'","").replace(',',' ')

'굳  ㅋ'

In [28]:
token =[]
for i in range(len(test_data)):
    tokens= str(test_data['tokens'][i]).replace('[','').replace(']','').replace("'","").replace(',',' ')
    token.append(tokens)
test_data['tokens'] = token
test_data

,id,document,label,tokens
0,6270596,굳 ㅋ,1,굳 ㅋ
1,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0,뭐 야 평점 나쁘 진 않 지만 점 짜리 더더욱 아니 잖아
2,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0,지루 하 지 않 은데 완전 막장 임 돈 주 고 보 기
3,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0,만 아니 었 어도 별 다섯 개 줬 을 텐데 왜 로 나와서 제...
4,7898805,음악이 주가 된 최고의 음악영화,1,음악 주 된 최고 음악 영화
...,...,...,...,...
48990,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,오랜만 평점 로 긴 했 네 ㅋㅋ 킹왕짱 쌈 뽕 영화 만났 습니...
48991,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0,의지 박약 이나 하 거 다 탈영 일단 주인공 김대희 닮 았 고...
48992,9072549,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0,그림 좋 고 완성도 높 았 지만 보 내내 불안 하 게 만든다
48993,5802125,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0,절대 봐서 안 될 영화 재미 없 고 기분 만 잡치 고 세트 ...


In [29]:
Y_train = list(train_data['label'])
print(Y_train[:3])

[0, 1, 0]


In [30]:
Y_test = list(test_data['label'])
print(Y_test[:3])

[1, 0, 0]


In [31]:
X_train = train_data['tokens']

In [32]:
X_test = test_data['tokens']

In [33]:
print(X_train)
print(X_test)

0                               아  더  빙  진짜  짜증  나  네요  목소리
1           흠  포스터  보고  초딩  영화  줄  오버  연기  조차  가볍  지  않  구나
2                                     너무  재  밓었다그래서보는것을추천한다
3                       교도소  이야기  구먼  솔직히  재미  없  다  평점  조정
4         사이몬페그  익살  스런  연기  돋보였  던  영화  스파이더맨  에서  늙  어...
                                ...                        
145786                                 인간  문제지  소  뭔  죄  인가
145787                                        평점  너무  낮  아서
145788              이게  뭐  요  한국인  거들먹거리  고  필리핀  혼혈  착하  다
145789                    청춘  영화  최고봉  방황  우울  했  던  날  자화상
145790                     한국  영화  최초  로  수간  하  내용  담긴  영화
Name: tokens, Length: 145791, dtype: object
0                                                     굳  ㅋ
1               뭐  야  평점  나쁘  진  않  지만  점  짜리  더더욱  아니  잖아
2                지루  하  지  않  은데  완전  막장  임  돈  주  고  보  기
3        만  아니  었  어도  별  다섯  개  줬  을  텐데  왜  로  나와서  제...
4                                     음악  주  된  최고  음악  영화
 

### Sklearn

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
#CountVectorizer 선언
count_vect = CountVectorizer()

In [43]:
# fit and transform
X_train_counts = count_vect.fit_transform(X_train)
# MultinomialNB 선언 and fit
clf = MultinomialNB().fit(X_train_counts , Y_train)

In [40]:
# 예측
print(clf.predict(count_vect.transform(X_train)))
# 확률
print(clf.predict_proba(count_vect.transform(X_train)))

[0 0 1 ... 0 1 1]
[[0.9431931  0.0568069 ]
 [0.83447793 0.16552207]
 [0.45527592 0.54472408]
 ...
 [0.94609562 0.05390438]
 [0.00889499 0.99110501]
 [0.19965262 0.80034738]]


In [49]:
# 예측
print(clf.predict(count_vect.transform(['영화 핵잼'])))
# 확률
print(clf.predict_proba(count_vect.transform(['영화 핵잼'])))

[1]
[[0.48223169 0.51776831]]
